In [2]:
class DataReader(object):
    def __init__(self, Nw = 15):
        self.Nw = Nw
        self.Feature_DF_List = deque([],maxlen=Nw)
        self.Component_Info_List = []
        
    
    def Ready(self, json_data):
        '''
        Set Component Info. List
        :param json_data
        :return: None
        '''
        for ind in range(len(json_data['COMPONENT'])):
            BLOCK_NO = json_data['COMPONENT'][ind]['BLOCK_NO']
            REFERENCE = json_data['COMPONENT'][ind]['REFERENCE']
            
            self.Component_Info_List.append((BLOCK_NO,REFERENCE))
            
        return None
    
    def Step(self, json_data):
        
        feature_dict = {}
        for ind, (BLOCK_NO,REFERENCE) in enumerate(self.Component_Info_List):
#             print('comp',ind)
            feature_pair = {}
            for AXIS in ['LENGTH','WIDTH']:
                feature_pair[AXIS] =  float(
                    json_data['COMPONENT'][ind]['INSP_TYPE_VALUE'][0]['PADOVERHANG'][AXIS])
            feature_dict[(BLOCK_NO,REFERENCE)] = feature_pair
        
        self.Feature_DF_List.append(pd.DataFrame(feature_dict).T)
        
        return None
    
    def PullFeatures(self, Axis, component_info, num = 1):
        feature_list = []
        for n in range(num):
            feature_list.append(float(self.Feature_DF_List[n-num][Axis][component_info]))
            
        return feature_list
    
    def PullLastDataframe(self):
        return self.Feature_DF_List[-1]
    

In [3]:
class RankingDiscriminator(object):
    def __init__(self,MaxOpNum = 20):
        self.MaxOpNum = MaxOpNum
        self.Component_Info_List = None
        self.Score_DF = None
    
    def Ready(self, Component_Info_List):
        self.Component_Info_List = Component_Info_List
        
        score_dict = {}
        for (BLOCK_NO,REFERENCE) in self.Component_Info_List:
            score_pair = {}
            for AXIS in ['LENGTH','WIDTH']:
                score_pair[AXIS] = 0
            score_dict[(BLOCK_NO,REFERENCE)] = score_pair
        
        self.Score_DF = pd.DataFrame(score_dict).T
        
        return None
        
    def Step(self, Feature_DF):
        for (BLOCK_NO,REFERENCE) in self.Component_Info_List:
            for AXIS in ['LENGTH','WIDTH']:
                self.Score_DF[AXIS][(BLOCK_NO,REFERENCE)] += float(Feature_DF[AXIS][(BLOCK_NO,REFERENCE)])
        
        return None
       
    def Rank(self, Current_State_DF):
        high_ranked_components = {'LENGTH':[],'WIDTH':[]}
        
        length_score_df = self.Score_DF['LENGTH'].drop(Current_State_DF[Current_State_DF.LENGTH == 'ON'].index)
        width_score_df = self.Score_DF['WIDTH'].drop(Current_State_DF[Current_State_DF.WIDTH == 'ON'].index)

        high_ranked_components['LENGTH'] = list(length_score_df.sort_values(axis=0, ascending=False)[:self.MaxOpNum].index)
        high_ranked_components['WIDTH'] = list(width_score_df.sort_values(axis=0, ascending=False)[:self.MaxOpNum].index)
        
        return high_ranked_components
    
    def ResetScore(self, Previous_State_DF, Current_State_DF):
        for (BLOCK_NO,REFERENCE) in self.Component_Info_List:
            for AXIS in ['LENGTH','WIDTH']:
                if Previous_State_DF[AXIS][(BLOCK_NO,REFERENCE)] == 'ON' and \
                    Current_State_DF[AXIS][(BLOCK_NO,REFERENCE)] == 'OFF':
                    self.Score_DF[AXIS][(BLOCK_NO,REFERENCE)] = 0
        
        return None

In [4]:
import numpy as np
from collections import deque
import scipy.stats as stats
from statsmodels.tsa.stattools import adfuller
from scipy.signal import savgol_filter
from math import floor
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd

from BOCD import bocd_meanNstd, NormalUnKnownMeanPrecision


def HotellingT2(window):
    '''
    Find indexes of anomaly using Hotelling T-square method
    calculate T2 score for each point & compare with calculated Upper-confidence-level.
    if exceed, anomaly

    :param window: data list
    :return: indexes of anomaly
    '''
    alpha = 0.2  #FIXME : must be larger value
    p = 1
    m = len(window)
    q = 2 * (m - 1) ** 2 / (3 * m - 4)

    UCL = (m - 1) ** 2 * stats.beta.isf(alpha, p / 2, (q - p - 1) / 2) / m

    mean = np.mean(window)
    V = np.array([])
    T2_list = []

    for ind in range(m - 1):
        V = np.append(V, window[ind + 1] - window[ind])

    S = np.array([0.5 * V.transpose() @ V / (m - 1)])

    for item in window:
        delta = np.array(item) - np.array(mean)
        T2 = delta * np.linalg.inv(np.array([S])) * delta
        T2_list.append(T2)

    anomaly = []
    for ind, value in enumerate(T2_list):
        if value > UCL:
            anomaly.append(ind)

    return anomaly

class Operator(object):    
    def __init__(self, Nw=15, Nomin=3, Nomax=20):
        '''
        :param Nw: window size for outlier
        :param Nomin: min window size for waiting Mounter-FeedBack (MFB) reflection
        :param Nomax: max window size for waiting Mounter-FeedBack (MFB) reflection
        '''
        
        self.LastCPD = 0  # index
        self.LastOP = 0  # index
        self.LastSubOP = 0  # index
        
        # status : two status considered i.e. 'noFBapplied' & 'FBapplied'
        self.status = 'noFBapplied'

        # window size
        self.Nw = Nw  # mentioned
        self.Nomin = Nomin  # mentioned
        self.Nomax = Nomax  # mentioned
        self.Nstd = Nw  # mentioned

        # data list
        
        self.data = []  # data list (FB applied list : collected data list if online production)

        # data window
        self.window = deque([], maxlen=self.Nw)  # Raw data list for calculating offset & for smoothing (Nw data)
        self.window_for_compare = deque([], maxlen=self.Nw)  # Smoothed data list to compare threshold & current state

        # Target & Threshold
        self.Target = 0
        self.offset_threshold = None

        # Operation
        self.sub_operation = False  # where or not already applied sub-operation, as sub-op applied only once right after CPD

        # waiting step for reflection
        self.wait_num = 0  # counting number for monitoring Mounter-FeedBack delay
        self.abrupt_num = 0  # number counting low p-value point for Abrupt CPD
        self.p = 0.05  # respective lower & upper p-value for Abrupt CPD
        self.estimator = None  # Gaussian KDE for Abrupt CPD => p-value calculated based on the Gaussian KDE regressed distribution

    def clear(self):
        self.LastCPD = 0  # index
        self.LastOP = 0  # index
        self.LastSubOP = 0  # index
        
        # status : two status considered i.e. 'noFBapplied' & 'FBapplied'
        self.status = 'noFBapplied'

        # data list
        self.data = []  # total data list (FB applied list : collected data list if online production)

        # data window
        self.window = deque([], maxlen=self.Nw)  # Raw data list for calculating offset & for smoothing (Nw data)
        self.window_for_compare = deque([], maxlen=self.Nw)  # Smoothed data list to compare threshold & current state

        # Target & Threshold
        self.Target = 0
        self.offset_threshold = None

        # Operation
        self.sub_operation = False  # where or not already applied sub-operation, as sub-op applied only once right after CPD

        # waiting step for reflection
        self.wait_num = 0  # counting number for monitoring Mounter-FeedBack delay
        self.abrupt_num = 0  # number counting low p-value point for Abrupt CPD
        self.estimator = None  # Gaussian KDE for Abrupt CPD => p-value calculated based on the Gaussian KDE regressed distribution

        return None
        
    def _run_CPD(self):
        '''
        Check if change has occurred in the latest (No + Nw) data
        :return: True/False, sequence point of CP
        '''
        # BOCD 가동
        TimeSequence = self.Nw + len(self.data) - self.LastOP  # window size for detecting change-point

        std = np.std(np.array(self.data[-TimeSequence:][:self.Nw]))  # Std calculation for BOCPD FIXME (:self.Nw)

        mu0 = 0  # Prior on Gaussian mean. (A parameter of normal-gamma prior distribution)
        gamma0 = 1  # (A parameter of normal-gamma prior distribution)
        alpha0 = 1  # (A parameter of normal-gamma prior distribution)
        beta0 = alpha0 * std ** 2  # "sqrt(beta/alpha) = std" (A parameter of normal-gamma prior distribution)

        hazard = 1 / 50.0  # Hazard survival function assumes probability to be a CP for each data point. #FIXME (1/20.0 ??)
        message = np.array([1])  # Iterative message calculated using previously collected data.

        Data_list = []
        RL_dist_list = []
        Temp_RL_dist = np.zeros((TimeSequence, TimeSequence))

        model = NormalUnKnownMeanPrecision(mu0, gamma0, alpha0, beta0)  # Data assumed be to a normal distribution.
        # Case : Both of mean and std unknown
        # BOCPD class called.

        RL = []  # Run-Length distribution initial list
        Start = []  # Estimated Change-Point starting point
        cp_list = [0]  # Detected Change-Point sequence saved list

        for ind, cont in enumerate(range(TimeSequence)):
            Data_list.append(self.data[-TimeSequence + ind])
            RL_dist, new_joint = bocd_meanNstd(data_list=Data_list, model=model,
                                               hazard=hazard, Message=message)

            message = new_joint  # each point sequential likelihood, or numerator of RL posterior distribution.
            RL_dist_list.append(RL_dist)
            Temp_RL_dist[ind, :ind + 1] = RL_dist[1:]  # RL distribution temporary saved as a element of a list

            RL.append(np.argmax(
                Temp_RL_dist[ind]))  # The highest probability corresponding sequence of RL posterior distribution
            Start.append((ind, ind - RL[-1]))

            if ind >= 1:
                if Start[-1][1] != Start[-2][1]:
                    if max(cp_list) > ind - RL[-1]:
                        cp_list = cp_list[:-1]  # Remove the lastly added Change-Point
                    else:
                        cp_list.append(ind - RL[-1])  # Change-point Detected and append into the list

        if not cp_list:  # If no FeedBack candidate found
            return False, None
        elif cp_list[-1] < self.Nw:  # Exclude a CP ahead of FeedBack operation applied sequence
            return False, None
        else:
            for cp in cp_list[1:]:  # Exclude initial point (the point '0')
                self.LastCPD = len(self.data) - TimeSequence + cp_list[-1]
            return True, cp_list[-1]

    def _is_within_threshold(self):
        '''
        Check if the smoothed data is inside the threshold
        :return: True/ False ('True' if data lies within threshold )
        '''
        is_withinThreshold = False

        # Outlier : Last Nw smoothed data must be the same sign
        '''
        check = 0
        for smoothed in self.window_for_compare:
            check += np.sign(smoothed)

        if np.abs(check) != self.Nw:
            is_withinThreshold = True  
        '''
        check = np.sign(self.window_for_compare[0])
        for smoothed in self.window_for_compare:
            if(check != np.sign(smoothed)):
                is_withinThreshold = True; break; #FIXME (Non-same-sign which is anomaly ??)

        # Outlier : Last Nw smoothed data must be out of threshold.
        for smoothed in self.window_for_compare:
            if np.abs(smoothed - self.Target) <= self.offset_threshold:
                is_withinThreshold = True

        return is_withinThreshold

    def _do_operation(self):
        '''
        Apply operation & Set operation delay
        OUTPUT : operation
        '''
        self.sub_operation = False

        # Exclude anomaly before calculating offset by using Hotelling's T2
        windowForAnomaly = self.window
        anomaly = HotellingT2(windowForAnomaly)
        for delete_ind, anomaly_ind in enumerate(anomaly):
            del windowForAnomaly[anomaly_ind - delete_ind] # windowForAnomaly : Anomaly excluded window 

        # Calculate offset & operation
        offset = np.mean(windowForAnomaly)
        
        # Append operation index
        self.LastOP = len(self.data)
        
        return self.Target - offset


    def _do_sub_operation(self):
        '''
        Apply sub-operation & Set sub-operation delay
        OUTPUT : operation
        '''
        self.sub_operation = True  # number of sub operations

        # Exclude anomaly before calculating offset by using Hotelling's T2
        windowForAnomaly = self.data[max(self.LastCPD, self.LastOP):]
        anomaly = HotellingT2(windowForAnomaly)
        for delete_ind, anomaly_ind in enumerate(anomaly):
            del windowForAnomaly[anomaly_ind - delete_ind] # windowForAnomaly : Anomaly excluded window 

        # Calculate offset & operation
        offset = np.mean(windowForAnomaly)
        
        # Append sub-operation index
        self.LastSubOP = len(self.data)
        
        self.clear()
        
        return self.Target - offset


    def step(self, feature):
        '''
        MFB Loop
        :param feature: offset data
        :return: None
        '''
        # START
        if self.status == 'noFBapplied':
            self._put_feature(feature)

            # Enough data ?
            if len(self.data) - self.LastCPD < self.Nw:
                return 0, None

            # Check if lies within calculated threshold. If not, do operation.
            if self._is_within_threshold():
                if len(self.data) == self.Nw:
                    self.clear()
                    return 0, 'IN'
                # If state is on target & stable, do one sseub-operation.
                if not self.sub_operation: # "False" if no sub-operation from last CPD point. (As only one sub-op applied for one ADF-test steady-state)
                    check_point = max(self.LastCPD, self.LastOP) #'op_list':not considered delay operation index list, 'CPD_list': BOCPD found CPD index list
                    if (len(self.data) - check_point) >= self.Nw:  # There must be enough data after Op. or CPD.
                        if adfuller(self.data[check_point:])[1] < 1e-6:  # Stable?
                            return self._do_sub_operation(), 'SUB'
                        if (len(self.data) - check_point) <= 3*self.Nw:
                            self.clear()
                            return 0, 'IN'
                return 0, None
            else:
                self.estimator = stats.gaussian_kde(self.window)  # Set Gaussian KDE for Abrupt CPD.   # FIXME : minimum of delay given, then calculate the estimator
                self.status = 'FBapplied'
                return self._do_operation(), 'MAIN' # SG-filter smoothing is not consider anomaly # FIXME
                
        # Waiting for FB to be applied.
        if self.status == 'FBapplied':
            self._put_feature(feature)
            self.wait_num += 1

            # Abrupt CPD : repeating low p-value point or waiting enough fires BOCD
            if self.estimator.integrate_box_1d(-np.Inf, feature) < self.p or \
                    self.estimator.integrate_box_1d(-np.Inf, feature) > 1 - self.p:
                self.abrupt_num += 1

            if self.wait_num >= self.Nomax:
                self._run_CPD()  # CPD fired
                self.wait_num = 0  # Reset self.wait_num
                self.abrupt_num = 0  # Reset self.abrupt_num
                self.status = 'noFBapplied'  # Reset self.status
                return 0, None

            elif self.abrupt_num >= self.Nomin:
                self._run_CPD()  # CPD fired
                self.wait_num = 0  # Reset self.wait_num
                self.abrupt_num = 0  # Reset self.abrupt_num
                self.status = 'noFBapplied'  # Reset self.status
                return 0, None

            else:
                return 0, None
            

        
    def _put_feature(self, feature):
        '''
        Get offset data
        :param feature: offset data
        :return: None
        '''
        # Append feature to data lists
        self.data.append(feature)  # Collected total data
        self.window.append(feature)  # Nw corresponding window size

        # Initialize the threshold
        if len(self.data) == self.Nstd:   # FIXME use once at the initial stage (put outside of the state stage)
            self.offset_threshold = 0.5 * np.std(self.data)

        # Data smoothing
        if len(self.data) >= self.Nw:
            savgol_result = savgol_filter(self.data[-self.Nw:], 2*floor((self.Nw-1)/2)+1, 3)
            self.window_for_compare = savgol_result  ## compare threshold with sg-filter regressed line
                    
        # Reset the threshold
        if (len(self.data) - self.LastCPD) % self.Nstd == 0:
            buffer = self.data[-self.Nstd:]
            STD = np.std(buffer)
            if 0.7 * self.offset_threshold > STD or self.offset_threshold < 0.7 * STD:
                self.offset_threshold = 0.5 * STD

In [5]:
class MounterSimulator(object):
    def __init__(self, json_data, NumList = 5):
        self.operation_buffer = np.array([[0,0]]*NumList)
        self.operation = copy.deepcopy(self.operation_buffer)
        self.delay = np.array([[[0],[0]]]*NumList)
        self.NumList = NumList 
        
        pcb_info = json_data
        pcb_info['COMPONENT'] = pcb_info['COMPONENT'][:self.NumList]
        self.PCB_INFO = pcb_info
        
        
    def PLANT(self, FEEDBACK_INFO, json_data):
        self.PCB_INFO = json_data
        for component_index in range(self.NumList):
            for axis_ind,AXIS in enumerate(['AdjustX','AdjustY']):
                if FEEDBACK_INFO['OFFSETLIST'][component_index][AXIS] != 0:
                    self.delay[component_index][axis_ind] = round(np.random.exponential(scale=2))+1
                    self.delay[component_index][axis_ind] = 1
                    self.operation_buffer[component_index][axis_ind] \
                        = FEEDBACK_INFO['OFFSETLIST'][component_index][AXIS]

                    
        self.delay -= 1
        
        for component_index in range(self.NumList):
            for axis_ind, AXIS in enumerate(['LENGTH','WIDTH']):
                if self.delay[component_index][axis_ind] == 0 :
                    self.operation[component_index][axis_ind] += float(
                        self.operation_buffer[component_index][axis_ind])
  
                self.PCB_INFO['COMPONENT'][component_index]['INSP_TYPE_VALUE'][0]['PADOVERHANG'][AXIS] \
                        = str(float(
                        self.PCB_INFO['COMPONENT'][component_index]['INSP_TYPE_VALUE'][0]['PADOVERHANG'][AXIS])
                        + self.operation[component_index][axis_ind])
      
        return self.PCB_INFO

In [6]:
class MounterFeedback(object):
    def __init__(self, MaxOpNum = 20, Nw = 15):
        self.MaxOpNum = MaxOpNum
        self.Nw = Nw
        
        self.Operator_DF = None
        self.Operation_DF = None
        self.Previous_State_DF = None
        self.Current_State_DF = None
        
        self.Component_Info_List = []
        
        self.DataReader = DataReader(Nw=Nw)
        self.RankingDiscriminator = RankingDiscriminator(MaxOpNum = MaxOpNum)
        
        self.state = 'noInitialShifting'
        
    def Ready(self, json_data):
        '''
        Initialize MFB algorithm & Ready for feedback
        :param json_data
        :return: None
        '''
        self.DataReader.Ready(json_data)
        self.RankingDiscriminator.Ready(self.DataReader.Component_Info_List)
        self.Component_Info_List = self.DataReader.Component_Info_List
        
        # Init operator_df & operation_df
        
        operator_dict = {}
        current_state_dict = {}
        previous_state_dict = {}
        operation_dict = {}
        
        for (BLOCK_NO,REFERENCE) in self.Component_Info_List:        
            operator_pair = {}
            current_state_pair = {}
            previous_state_pair = {}
            operation_pair = {}
            for AXIS in ['LENGTH','WIDTH']:
                operator_pair[AXIS] = Operator(Nw=15, Nomin=3, Nomax=20)
                current_state_pair[AXIS] = 'OFF'
                previous_state_pair[AXIS] = 'OFF'
                operation_pair[AXIS] = (0, None)
                
            operator_dict[(BLOCK_NO,REFERENCE)] = operator_pair
            current_state_dict[(BLOCK_NO,REFERENCE)] = current_state_pair
            previous_state_dict[(BLOCK_NO,REFERENCE)] = previous_state_pair
            operation_dict[(BLOCK_NO,REFERENCE)] = operation_pair
            
        self.Operator_DF = pd.DataFrame(operator_dict).T
        self.Current_State_DF = pd.DataFrame(current_state_dict).T
        self.Previous_State_DF = pd.DataFrame(previous_state_dict).T
        self.Operation_DF = pd.DataFrame(operation_dict).T
        
        
        
    def ReturnFeedbackInfo(self):
        FEEDBACK_INFO = {"FeedbackType": "PLACEDLOCATIONOFFSET",
                         "SeqID": json_data["ASST_SEQNO"],
                         "Time": json_data["END_DATE_TIME"],
                         "MachineName": json_data["MACHINE_NM"],
                         "LaneNo": json_data["LANE_NO"],
                         "Target": '1',
                         "NumList": len(json_data["COMPONENT"]),
                         "OFFSETLIST": []}
        

        for (BLOCK_NO,REFERENCE) in self.Component_Info_List:  
            FEEDBACK_INFO["OFFSETLIST"].append(
                {'BLOCK_NO': BLOCK_NO, 'REFERENCE': REFERENCE,
                 'AdjustX': self.Operation_DF['LENGTH'][(BLOCK_NO,REFERENCE)][0],
                 'AdjustY': self.Operation_DF['WIDTH'][(BLOCK_NO,REFERENCE)][0]})
                
        return FEEDBACK_INFO
        
                
    def Step(self, json_data):
        '''
        param json_data
        return: feedback information as dictionary
        '''
        self.DataReader.Step(json_data)
        
        self.RankingDiscriminator.Step(self.DataReader.PullLastDataframe())
        
        high_ranked_components = self.RankingDiscriminator.Rank(self.Current_State_DF)
        
        
        if len(self.DataReader.Feature_DF_List) < self.Nw :
            return self.ReturnFeedbackInfo()
        
        for AXIS in ['LENGTH','WIDTH']:
            for (BLOCK_NO,REFERENCE) in high_ranked_components[AXIS]:
                self.Current_State_DF[AXIS][(BLOCK_NO,REFERENCE)] = 'ON'
                
                
        for AXIS in ['LENGTH','WIDTH']:
            for (BLOCK_NO,REFERENCE) in self.Component_Info_List:
                if self.Current_State_DF[AXIS][(BLOCK_NO,REFERENCE)] == 'ON':
                    if self.Previous_State_DF[AXIS][(BLOCK_NO,REFERENCE)] == 'OFF':
#                         if (BLOCK_NO,REFERENCE)==('1','C10'):
#                             print('asdasdasdaasfads')
                        ResentData = self.DataReader.PullFeatures(
                            AXIS,(BLOCK_NO,REFERENCE),num=self.Nw)[:-1]
                        self.Operator_DF[AXIS][(BLOCK_NO,REFERENCE)].data += ResentData
                        self.Operator_DF[AXIS][(BLOCK_NO,REFERENCE)].window += ResentData
                        
                        operation = self.Operator_DF[AXIS][(BLOCK_NO,REFERENCE)].step(
                            self.DataReader.PullFeatures(AXIS,(BLOCK_NO,REFERENCE))[0])
                        
                        self.Operation_DF[AXIS][(BLOCK_NO,REFERENCE)] = operation
                    
                    else:
                        operation = self.Operator_DF[AXIS][(BLOCK_NO,REFERENCE)].step(
                            self.DataReader.PullFeatures(AXIS,(BLOCK_NO,REFERENCE))[0])
                        
                        self.Operation_DF[AXIS][(BLOCK_NO,REFERENCE)] = operation
                        
#                     if (BLOCK_NO,REFERENCE)==('1','C10'):
#                         print('^^')
#                         print(self.Operator_DF[AXIS][(BLOCK_NO,REFERENCE)].data)
                        
                else:
                    self.Operation_DF[AXIS][(BLOCK_NO,REFERENCE)] = (0, None)
        
        
        self.Previous_State_DF = self.Current_State_DF.copy()
        
        self.RankingDiscriminator.ResetScore(self.Previous_State_DF, self.Current_State_DF)
        
        for AXIS in ['LENGTH','WIDTH']:
            for (BLOCK_NO,REFERENCE) in self.Component_Info_List:
                if self.Operation_DF[AXIS][(BLOCK_NO,REFERENCE)][1] == 'SUB' or \
                  self.Operation_DF[AXIS][(BLOCK_NO,REFERENCE)][1] == 'IN':
                    self.Current_State_DF[AXIS][(BLOCK_NO,REFERENCE)] = 'OFF'
        
        
        return self.ReturnFeedbackInfo()


In [8]:
if __name__ == '__main__':
    import os
    import json
    import copy
    
    Data_temp = pd.read_csv("C:/Users/T5402/Downloads/n1_test_500k.txt", sep=',')

    
    json_directory_path = 'C:/Users/T5402/Desktop/merge_samples2'
    json_file_list = os.listdir(json_directory_path)
    
    f = open(json_directory_path+'/'+json_file_list[0],'r')
    json_data = dict(json.load(f))
    
    
    NumList = 4
    TimeSequence = 450
    
    MFB = MounterFeedback(MaxOpNum=1)
    MFB.Ready(json_data)
    mounter = MounterSimulator(json_data, NumList = NumList )
    
    raw = []
    data1 = []
    main = []
    sub = []
    
    for i, file_name in enumerate(json_file_list[50:TimeSequence]):
        f = open(json_directory_path+'/'+file_name,'r')
        json_data = dict(json.load(f))
        raw.append(float(json_data['COMPONENT'][1]['INSP_TYPE_VALUE'][0]['PADOVERHANG']['WIDTH']))

        if i > 10 and MFB.Operation_DF['WIDTH'][('1','C10')][1] !=  None\
            and MFB.Operation_DF['WIDTH'][('1','C10')][1] !=  'IN':
            print('====================\npcb seq no',i)
            print(MFB.Current_State_DF['WIDTH'][('1','C10')],
                  '\n',MFB.Operation_DF['WIDTH'][('1','C10')])

        json_data = mounter.PLANT(FEEDBACK_INFO, json_data)
        FEEDBACK_INFO = MFB.Step(json_data)
        
        if MFB.Operation_DF['WIDTH'][('1','C10')][1] == 'MAIN':
            main.append(i)
        if MFB.Operation_DF['WIDTH'][('1','C10')][1] == 'SUB':
            sub.append(i)
            
        data1.append(float(
            mounter.PCB_INFO['COMPONENT'][1]['INSP_TYPE_VALUE'][0]['PADOVERHANG']['WIDTH']))

    
    RMS_before_FB = round(np.sqrt(sum((np.array(raw) - np.zeros(len(raw))) ** 2) / len(raw)),2)
    RMS_after_FB = round(np.sqrt(sum((np.array(data1) - np.zeros(len(data1))) ** 2) / len(data1)),2)
    
    print('RMS_before_FB\t: ',RMS_before_FB)
    print('RMS_after_FB\t: ',RMS_after_FB)
    
    fit, (ax1) = plt.subplots(nrows=1, ncols=1, figsize=(20, 5))
    ax1.grid(True)
    ax1.plot(raw[:], 'grey')
    ax1.plot(data1[:], 'blue')
    ax1.axhline(0, linestyle='--', color='grey')
    for i in main:
        ax1.axvline(i, color = 'green', linewidth = 1, linestyle = '--')
    for i in sub:
        ax1.axvline(i, color = 'red', linewidth = 1, linestyle = '--')

NameError: name 'FEEDBACK_INFO' is not defined

In [41]:
print(str('sdf'))

sdf


In [7]:
if __name__ == '__main__':
    import os
    import json
    import copy
    
    Data_temp = pd.read_csv("C:/Users/T5402/Downloads/n1_test_500k.txt", sep=',')

    Pad_Temp = Data_temp[(Data_temp.Array_index == 2) & (Data_temp.Component_Name == 'R12')]
    PadOff_X = np.array(Pad_Temp.PAD_Length_offset / 1000.0)[500:]
    PadOff_Y = np.array(Pad_Temp.PAD_Width_offset / 1000.0)[500:]
    
    
    json_directory_path = 'C:/Users/T5402/Desktop/merge_samples2'
    json_file_list = os.listdir(json_directory_path)
    
    f = open(json_directory_path+'/'+json_file_list[0],'r')
    json_data = copy.deepcopy(json.load(f))
    
    
    for i in range(1000):
        for j in range(4):
            json_data['COMPONENT'][j]['INSP_TYPE_VALUE'][0]['PADOVERHANG']['LENGTH'] = str(PadOff_X[i]+np.random.normal(0,10))
            json_data['COMPONENT'][j]['INSP_TYPE_VALUE'][0]['PADOVERHANG']['WIDTH'] = str(PadOff_Y[i]+np.random.normal(0,10))
        with open('C:/Users/T5402/Desktop/merge_samples2/merge_sample'+str(i)+'.json', 'w') as make_file:
            json.dump(json_data, make_file, indent="\t")